# Phase-based analysis of microbiome rhythmicity

This notebook provides executable Python code to perform **phase-based analyses**
of microbial oscillations detected by the Non-Parametric Period Detection (NPPD) method.  


## 1. Hierarchical clustering of phase profiles

We clustered genomes based on their circadian phase trajectories using a phase-locking–based distance metric.  
The function below takes a phase time-series matrix `phase_df` (rows = MAGs, columns = time points, values = phase θ in radians) and returns cluster labels.



In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.metrics import silhouette_score

# ---------- helpers ----------
def wrap_mpi_pi(x):
    """Wrap angles to (-π, π]."""
    return (np.asarray(x, float) + np.pi) % (2 * np.pi) - np.pi


def pair_plv_distance(phi_i, phi_j, valid_i=None, valid_j=None, mod_pi=False):
    """
    Phase-locking–based distance between two phase time series.

    Parameters
    ----------
    phi_i, phi_j : array-like
        Phase time series (radians).
    valid_i, valid_j : array-like of bool, optional
        Masks indicating valid time points (True = use).
        If None, all finite values are used.
    mod_pi : bool, default False
        If True, treat phases separated by π as equivalent
        (use 2Δφ instead of Δφ).

    Returns
    -------
    d : float
        Distance in [0, 1], where 0 indicates perfect phase locking.
    """
    phi_i = np.asarray(phi_i, float)
    phi_j = np.asarray(phi_j, float)

    if valid_i is None:
        valid_i = np.isfinite(phi_i)
    if valid_j is None:
        valid_j = np.isfinite(phi_j)

    m = valid_i & valid_j & np.isfinite(phi_i) & np.isfinite(phi_j)
    if m.sum() == 0:
        return 1.0

    dphi = wrap_mpi_pi(phi_i[m] - phi_j[m])
    z = np.exp(1j * (2.0 * dphi if mod_pi else dphi))
    # distance based on the argument of the mean vector
    return (1.0 - np.cos(np.angle(np.mean(z)))) / 2.0


def pairwise_plv_distance_matrix(phase_df, valid_mask=None, mod_pi=False):
    """
    Compute a pairwise distance matrix between all rows in phase_df
    using the PLV-based phase distance.

    Parameters
    ----------
    phase_df : pandas.DataFrame
        Rows = taxa/MAGs, columns = time points, values = phase (radians).
    valid_mask : array-like of bool, shape (n_rows, n_time), optional
        Mask of valid time points for each row.
    mod_pi : bool, default False
        Passed to pair_plv_distance.

    Returns
    -------
    D : pandas.DataFrame
        Symmetric distance matrix (n_rows × n_rows).
    """
    labels = phase_df.index
    n = len(labels)
    D = np.zeros((n, n), float)

    for i in range(n):
        phi_i = phase_df.iloc[i].values
        vi = None if valid_mask is None else valid_mask[i]
        for j in range(i + 1, n):
            phi_j = phase_df.iloc[j].values
            vj = None if valid_mask is None else valid_mask[j]
            d = pair_plv_distance(phi_i, phi_j, vi, vj, mod_pi=mod_pi)
            D[i, j] = D[j, i] = d

    return pd.DataFrame(D, index=labels, columns=labels)


# ---------- main clustering function ----------
def phase_based_clustering(
    phase_df,
    valid_mask=None,
    n_clusters=None,
    auto_select=True,
    k_min=2,
    k_max=10,
    linkage_method="average",
    metric="precomputed",
    random_state=None,
    verbose=False,
):
    """
    Cluster taxa/MAGs based on their phase time series.

    Parameters
    ----------
    phase_df : pandas.DataFrame
        Rows = taxa/MAGs, columns = time points, values = phase (radians).
    valid_mask : array-like of bool, shape (n_rows, n_time), optional
        Mask of valid time points (True = use). If None, all finite phases
        are used for each row.
    n_clusters : int, optional
        Number of clusters. Required if auto_select=False.
    auto_select : bool, default True
        If True, select n_clusters in [k_min, k_max] that maximizes
        the silhouette score.
    k_min, k_max : int, default 2, 10
        Range of cluster numbers to consider when auto_select=True.
    linkage_method : str, default "average"
        Linkage method for hierarchical clustering.
    metric : str, default "precomputed"
        Metric used in silhouette_score; should be "precomputed"
        since we use a distance matrix.
    random_state : unused placeholder, for API compatibility.
    verbose : bool, default False
        If True, print the selected number of clusters and silhouette score.

    Returns
    -------
    labels : pandas.Series
        Cluster labels (1..K) indexed by phase_df.index.
    linkage_matrix : ndarray
        Linkage matrix returned by scipy.cluster.hierarchy.linkage,
        useful for plotting dendrograms.
    """
    # 1. distance matrix
    D = pairwise_plv_distance_matrix(phase_df, valid_mask, mod_pi=False)
    Y = squareform(D.values, checks=False)

    # 2. hierarchical clustering
    linkage_matrix = linkage(Y, method=linkage_method)

    # 3. select number of clusters
    if auto_select:
        best_k = None
        best_score = -np.inf
        best_labels = None

        max_k = min(k_max, len(phase_df))
        for k in range(k_min, max_k + 1):
            labels_k = fcluster(linkage_matrix, t=k, criterion="maxclust")
            if len(set(labels_k)) < 2:
                continue
            try:
                score = silhouette_score(D.values, labels_k, metric=metric)
            except Exception:
                continue
            if score > best_score:
                best_score = score
                best_k = k
                best_labels = labels_k

        if best_labels is None:
            raise RuntimeError("Failed to determine optimal cluster number.")
        if verbose:
            print(f"Optimal n_clusters = {best_k} (silhouette = {best_score:.3f})")
        labels = best_labels
    else:
        if n_clusters is None:
            raise ValueError("n_clusters must be specified when auto_select=False")
        labels = fcluster(linkage_matrix, t=n_clusters, criterion="maxclust")

    labels = pd.Series(labels, index=phase_df.index, name="cluster")
    return labels, linkage_matrix

In [6]:
# Example use
phase_df = pd.read_csv('input_for_PhaseAnalysis/MAG_test_phase.csv', header=0, index_col=0)
labels, Z = phase_based_clustering(phase_df, auto_select=True)
phase_df_with_cluster = phase_df.assign(cluster=labels)

## 2. Kuramoto order parameter

The Kuramoto order parameter provides a quantitative measure of phase coherence within each genome cluster.  
Given genome-resolved phase trajectories (phase_df) and assigned cluster labels, the function below computes:

- **$R_c(t)$** — the Kuramoto order parameter for cluster *c* at time *t*, ranging from 0 (desynchronized) to 1 (perfect synchrony).  
- **$\phi_c(t)$** — the mean phase of cluster *c* at each time point.

The function requires only two inputs—`phase_df` and `labels`—and returns two DataFrames:  
`R_df` (cluster × time) and `phi_df` (cluster × time).

In [4]:
import numpy as np
import pandas as pd

def kuramoto_order_parameter(phase_df, labels):
    """
    Compute the Kuramoto order parameter R_c(t) and mean phase ψ_c(t)
    for each cluster c from phase time-series data.

    Parameters
    ----------
    phase_df : pandas.DataFrame
        Rows = units (e.g., MAGs), columns = time points, values = phase θ (radians).
    labels : pandas.Series or array-like
        Cluster labels for each unit. If a Series, its index should match phase_df.index.

    Returns
    -------
    R_df : pandas.DataFrame
        Kuramoto order parameter R_c(t) for each cluster (rows) and time point (columns).
    phi_df : pandas.DataFrame
        Mean phase ψ_c(t) ∈ [0, 2π) for each cluster and time point.
    """
    # Align labels to phase_df
    if isinstance(labels, pd.Series):
        labels = labels.reindex(phase_df.index)
    else:
        labels = pd.Series(labels, index=phase_df.index)

    time_vals = phase_df.columns
    clusters = np.sort(labels.unique())

    R_dict = {}
    phi_dict = {}

    phases = phase_df.to_numpy(dtype=float)

    for c in clusters:
        idx = labels == c
        Phi_c = phases[idx, :]  # shape: (Nc, T)
        Nc, T = Phi_c.shape

        R_c = np.full(T, np.nan, float)
        phi_c = np.full(T, np.nan, float)

        for ti in range(T):
            phi_t = Phi_c[:, ti]
            valid = np.isfinite(phi_t)
            if not np.any(valid):
                continue
            z = np.exp(1j * phi_t[valid])
            m = z.mean()
            R_c[ti] = np.abs(m)
            phi_c[ti] = np.angle(m) % (2 * np.pi)

        R_dict[c] = R_c
        phi_dict[c] = phi_c

    R_df = pd.DataFrame(R_dict, index=time_vals).T
    phi_df = pd.DataFrame(phi_dict, index=time_vals).T

    return R_df, phi_df

In [9]:
# Example use
R_df, phi_df = kuramoto_order_parameter(phase_df, labels)